tgb - 4/18/2019 
- We will train unconstrained (U), loss-constrained (L) and architecture-constrained (A) neural networks trained on the 8-column +0K experiment and validated on the same experiment. The loss-constrained networks will have varying architectures to see how MSE and energy conservation performances vary with the importance given to each in the loss function.  
  
Notebook 009 follows the notebook 005 that predicts:
***
[PHQ, PHCLDLIQ, PHCLDICE, TPHYSTND, QRL, QRS, DTVKE, FSNT, FSNS, FLNT, FLNS, PRECT, PRECTEND, PRECST, PRECSTEN] as a function of:  
[QBP, QCBP, QIBP, TBP, VBP, Qdt_adiabatic, QCdt_adiabatic, QIdt_adiabatic, Tdt_adiabatic, Vdt_adiabatic, PS, SOLIN, SHFLX, LHFLX] 

# 1) Load modules and create training/validation data generators

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
import tensorflow as tf
import tensorflow.math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()
TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
#PREFIXDS = '8col009_ds1_'
PREFIXDS = PREFIX
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

SyntaxError: invalid syntax (imports.py, line 24)

tgb - 4/18/2019 - Used preprocessed data calculated in notebook 009 @ https://github.com/tbeucler/CBRAIN-CAM/blob/master/notebooks/tbeucler_devlog/009_Generalization_Climate_Change_8col.ipynb

In [3]:
scale_dict = load_pickle('./nn_config/scale_dicts/009_Wm2_scaling.pkl')
in_vars = load_pickle('./nn_config/scale_dicts/009_Wm2_in_vars.pkl')
out_vars = load_pickle('./nn_config/scale_dicts/009_Wm2_out_vars.pkl')
dP = load_pickle('./nn_config/scale_dicts/009_Wm2_dP.pkl')

train_gen = DataGenerator(
    data_fn = TRAINDIR+PREFIXDS+'train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+PREFIX+'norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)
valid_gen = DataGenerator(
    data_fn = TRAINDIR+PREFIX+'valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+PREFIX+'norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=False
)

# 2) Build neural networks in a loop
tgb - 4/18/2019 - https://stackoverflow.com/questions/52320059/creating-a-new-sequential-model-inside-a-for-loop-using-keras  
tgb - 4/24/2019 - Edited for data-scarce network

In [4]:
hsav = {}
hsav[ep] = hist.history

# 5) Save model
path = TRAINDIR+'HDF5_DATA/NNL'+str(alpha)+'DS1_'+str(ep)+'.h5'
NN.save(path)
print('NN saved in ',path)

RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().

In [4]:
#alpha_array = [0,0.01,0.25,0.5,0.75,0.99,1] # Loop over weight given to MSE and conservation constraints
alpha_array = [0]
Nep = 100; hsav = {};

for alpha in alpha_array:
    NN = {}
    print('alpha = ',str(alpha),' and NN is ',NN)
    graph = tf.Graph()
    with tf.Session(graph=graph):
        
        # 1) Create model
        # Unconstrained model with 5 dense layers (Notebook 009)
        inpU = Input(shape=(304,))
        densout = Dense(512, activation='linear')(inpU)
        densout = LeakyReLU(alpha=0.3)(densout)
        for i in range (4):
            densout = Dense(512, activation='linear')(densout)
            densout = LeakyReLU(alpha=0.3)(densout)
        densout = Dense(218, activation='linear')(densout)
        out_layer = LeakyReLU(alpha=0.3)(densout)
        NN = tf.keras.models.Model(inpU, out_layer)
        print('NN is ',NN.summary())
        
        # 2) Define loss
        al = alpha/4 # Weight given to each residual
        Loss = WeakLoss(inpU, inp_div=train_gen.input_transform.div,
                            inp_sub=train_gen.input_transform.sub,
                            norm_q=scale_dict['PHQ'],
                            hyai=hyai, hybi=hybi, name='loss',
                            alpha_mass=al, alpha_ent=al,
                            alpha_lw=al, alpha_sw=al)
        
        # 3) Compile model
        NN.compile(tf.keras.optimizers.RMSprop(), loss=Loss, metrics=[mse])
        
        # 4) Train model
        # Save history following https://stackoverflow.com/questions/49969006/save-and-load-keras-callbacks-history
#         for ep in range(20):
        hist = NN.fit_generator(train_gen, epochs=Nep)
        hsav = hist.history

        # 5) Save model
        path = TRAINDIR+'HDF5_DATA/NNL'+str(alpha)+'DS1.h5'
        NN.save(path)
        print('NN saved in ',path)

alpha =  0  and NN is  {}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 304)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               156160    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)          

tgb - 5/1/2019 - Modified to create NNA network with 0.01=alpha loss function

In [ ]:
Nep = 20
# Repeat for architecture-constrained network
graph = tf.Graph()
with tf.Session(graph=graph):

    # 1) Create model
    # Unconstrained model with 5 dense layers (Notebook 009)
    inpC = Input(shape=(304,))
    densout = Dense(512, activation='linear')(inpC)
    densout = LeakyReLU(alpha=0.3)(densout)
    for i in range (4):
        densout = Dense(512, activation='linear')(densout)
        densout = LeakyReLU(alpha=0.3)(densout)
    densout = Dense(214, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
    surfout = SurRadLayer(
        inp_div=train_gen.input_transform.div,
        inp_sub=train_gen.input_transform.sub,
        norm_q=scale_dict['PHQ'],
        hyai=hyai, hybi=hybi
    )([inpC, densout])
    massout = MassConsLayer(
        inp_div=train_gen.input_transform.div,
        inp_sub=train_gen.input_transform.sub,
        norm_q=scale_dict['PHQ'],
        hyai=hyai, hybi=hybi
    )([inpC, surfout])
    enthout = EntConsLayer(
        inp_div=train_gen.input_transform.div,
        inp_sub=train_gen.input_transform.sub,
        norm_q=scale_dict['PHQ'],
        hyai=hyai, hybi=hybi
    )([inpC, massout])
    NNA = tf.keras.models.Model(inpC, enthout)
    print(NNA.summary())
    
    al = 0.01/4
    Loss = WeakLoss(inpC, inp_div=train_gen.input_transform.div,
                            inp_sub=train_gen.input_transform.sub,
                            norm_q=scale_dict['PHQ'],
                            hyai=hyai, hybi=hybi, name='loss',
                            alpha_mass=al, alpha_ent=al,
                            alpha_lw=al, alpha_sw=al)
    
    # 2) Compile model
    NNA.compile(tf.keras.optimizers.RMSprop(), loss=Loss, metrics=[mse])

    # 3) Train model
    NNA.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen)

    # 4) Save model
    path = TRAINDIR+'HDF5_DATA/NNA0.01.h5'
    NNA.save(path)
    print('NN saved in ',path)

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 304)          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 512)          156160      input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 512)          0           dense[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512)          262656      leaky_re_lu[0][0]                
_____________________________________

tgb - 4/23/2019 - Repeat unconstrained and architecture-constrained examples in multiple linear regression mode to compare performances.  
To make the models linear, simply eliminate the leaky reLU layers.

In [3]:
Nep = 20
alpha = 0

NN = {}
print('alpha = ',str(alpha),' and NN is ',NN)
graph = tf.Graph()
with tf.Session(graph=graph):

    # 1) Create model
    # Unconstrained model with 5 dense layers (Notebook 009)
    inpU = Input(shape=(304,))
    densout = Dense(512, activation='linear')(inpU)
    for i in range (4):
        densout = Dense(512, activation='linear')(densout)
    out_layer = Dense(218, activation='linear')(densout)
    NN = tf.keras.models.Model(inpU, out_layer)
    print('NN is ',NN.summary())

    # 2) Define loss
    al = alpha/4 # Weight given to each residual
    Loss = WeakLoss(inpU, inp_div=train_gen.input_transform.div,
                        inp_sub=train_gen.input_transform.sub,
                        norm_q=scale_dict['PHQ'],
                        hyai=hyai, hybi=hybi, name='loss',
                        alpha_mass=al, alpha_ent=al,
                        alpha_lw=al, alpha_sw=al)

    # 3) Compile model
    NN.compile(tf.keras.optimizers.RMSprop(), loss=Loss, metrics=[mse])

    # 4) Train model
    NN.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen)

    # 5) Save model
    path = TRAINDIR+'HDF5_DATA/MLRL'+str(alpha)+'.h5'
    NN.save(path)
    print('MLR saved in ',path)

alpha =  0  and NN is  {}
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 304)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               156160    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
____________________________________________

tgb - 4/23/2019 - Repeat the operation of linearing the neural network in the architecture-constrained case

In [4]:
# Repeat for architecture-constrained network
graph = tf.Graph()
with tf.Session(graph=graph):

    # 1) Create model
    # Unconstrained model with 5 dense layers (Notebook 009)
    inpC = Input(shape=(304,))
    densout = Dense(512, activation='linear')(inpC)
    for i in range (4):
        densout = Dense(512, activation='linear')(densout)
    densout = Dense(214, activation='linear')(densout)
    surfout = SurRadLayer(
        inp_div=train_gen.input_transform.div,
        inp_sub=train_gen.input_transform.sub,
        norm_q=scale_dict['PHQ'],
        hyai=hyai, hybi=hybi
    )([inpC, densout])
    massout = MassConsLayer(
        inp_div=train_gen.input_transform.div,
        inp_sub=train_gen.input_transform.sub,
        norm_q=scale_dict['PHQ'],
        hyai=hyai, hybi=hybi
    )([inpC, surfout])
    enthout = EntConsLayer(
        inp_div=train_gen.input_transform.div,
        inp_sub=train_gen.input_transform.sub,
        norm_q=scale_dict['PHQ'],
        hyai=hyai, hybi=hybi
    )([inpC, massout])
    NNA = tf.keras.models.Model(inpC, enthout)
    print(NNA.summary())

    # 2) Compile model
    NNA.compile(tf.keras.optimizers.RMSprop(), loss=mse, metrics=[mse])

    # 3) Train model
    NNA.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen)

    # 4) Save model
    path = TRAINDIR+'HDF5_DATA/MLRA.h5'
    NNA.save(path)
    print('MLR saved in ',path)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 304)          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 512)          156160      input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512)          262656      dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 512)          262656      dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (D

Epoch 20/20
41376/41376 [==============================] - 717s 17ms/step - loss: 290.9178 - mean_squared_error: 290.9177 - val_loss: 296.3175 - val_mean_squared_error: 296.3186
MLR saved in  /local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/MLRA.h5


## 3) Calculate statistics and residuals in a loop

## 3.1) Load models to test statistics
tgb - 4/19/2019 - Debugging

In [8]:
# config_fn = '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local_PostProc.yml'
# data_fn = '/local/Tom.Beucler/SPCAM_PHYS/8col009_01_valid.nc'
# dict_lay = {'SurRadLayer':SurRadLayer,'MassConsLayer':MassConsLayer,'EntConsLayer':EntConsLayer}

# %cd $TRAINDIR/HDF5_DATA
# !ls
# dict_lay = {'SurRadLayer':SurRadLayer,'MassConsLayer':MassConsLayer,'EntConsLayer':EntConsLayer}
# alpha = 0
# path = TRAINDIR+'HDF5_DATA/NNL'+str(alpha)+'.h5'
# NN = load_model(path,custom_objects=dict_lay)
# md = ModelDiagnostics(NN,config_fn,data_fn)
    
# # 3) Calculate statistics and save in pickle file
# md.compute_stats(niter=5)
# path = TRAINDIR+'HDF5_DATA/NNL'+str(alpha)+'md.pkl'
# pickle.dump(md.stats,open(path,'wb'))
# print('Stats are saved in ',path)

# # 4) Calculate budget residuals and save in pickle file
# md.compute_res(niter=5)
# path = TRAINDIR+'HDF5_DATA/NNL'+str(alpha)+'res.pkl'
# pickle.dump(md.res,open(path,'wb'))
# print('Budget residuals are saved in ',path)

## 3.2) Statistics

In [3]:
# config_fn = '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local_PostProc.yml'
# data_fn = '/local/Tom.Beucler/SPCAM_PHYS/8col009_01_valid.nc'
# dict_lay = {'SurRadLayer':SurRadLayer,'MassConsLayer':MassConsLayer,'EntConsLayer':EntConsLayer}

# for alpha in alpha_array:
    
#     # 1) Load model
#     path = TRAINDIR+'HDF5_DATA/NNL'+str(alpha)+'.h5'
#     NN = load_model(path,custom_objects=dict_lay)
    
#     # 2) Define model diagnostics object
#     md = ModelDiagnostics(NN,config_fn,data_fn)
    
#     # 3) Calculate statistics and save in pickle file
#     md.compute_stats()
#     path = TRAINDIR+'HDF5_DATA/NNL'+str(alpha)+'md.pkl'
#     pickle.dump(md.stats,open(path,'wb'))
#     print('Stats are saved in ',path)
    
#     # 4) Calculate budget residuals and save in pickle file
#     md.compute_res()
#     path = TRAINDIR+'HDF5_DATA/NNL'+str(alpha)+'res.pkl'
#     pickle.dump(md.res,open(path,'wb'))
#     print('Budget residuals are saved in ',path)

### Statistics of architecture-constrained neural network

In [ ]:
# config_fn = '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local_PostProc.yml'
# data_fn = '/local/Tom.Beucler/SPCAM_PHYS/8col009_01_valid.nc'
# dict_lay = {'SurRadLayer':SurRadLayer,'MassConsLayer':MassConsLayer,'EntConsLayer':EntConsLayer}

# NN = {}; MD = {};
# # 1) Load model
# path = TRAINDIR+'HDF5_DATA/NNA.h5'
# NN = load_model(path,custom_objects=dict_lay)

# # 2) Define model diagnostics object
# md = ModelDiagnostics(NN,config_fn,data_fn)

# # 3) Calculate statistics and save in pickle file
# # md.compute_stats()
# # path = TRAINDIR+'HDF5_DATA/NNAmd.pkl'
# # pickle.dump(md.stats,open(path,'wb'))
# # print('Stats are saved in ',path)

# # 4) Calculate budget residuals and save in pickle file
# md.compute_res()
# path = TRAINDIR+'HDF5_DATA/NNAres.pkl'
# pickle.dump(md.res,open(path,'wb'))
# print('Budget residuals are saved in ',path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


### Statistics of multiple linear regressions  
tgb - 4/23/2019

In [2]:
config_fn = '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local_PostProc.yml'
data_fn = '/local/Tom.Beucler/SPCAM_PHYS/8col009_01_valid.nc'
dict_lay = {'SurRadLayer':SurRadLayer,'MassConsLayer':MassConsLayer,'EntConsLayer':EntConsLayer}

# 1) Load model
path = TRAINDIR+'HDF5_DATA/MLRL0.h5'
NN = load_model(path,custom_objects=dict_lay)

# 2) Define model diagnostics object
md = ModelDiagnostics(NN,config_fn,data_fn)

# 3) Calculate statistics and save in pickle file
# md.compute_stats()
# path = TRAINDIR+'HDF5_DATA/MLRL0md.pkl'
# pickle.dump(md.stats,open(path,'wb'))
# print('Stats are saved in ',path)

# 4) Calculate budget residuals and save in pickle file
md.compute_res()
path = TRAINDIR+'HDF5_DATA/MLRL0res.pkl'
pickle.dump(md.res,open(path,'wb'))
print('Budget residuals are saved in ',path)

NN = {}; md = {};

# 1) Load model
path = TRAINDIR+'HDF5_DATA/MLRA.h5'
NN = load_model(path,custom_objects=dict_lay)

# 2) Define model diagnostics object
md = ModelDiagnostics(NN,config_fn,data_fn)

# 3) Calculate statistics and save in pickle file
md.compute_stats()
path = TRAINDIR+'HDF5_DATA/MLRAmd.pkl'
pickle.dump(md.stats,open(path,'wb'))
print('Stats are saved in ',path)

# 4) Calculate budget residuals and save in pickle file
md.compute_res()
path = TRAINDIR+'HDF5_DATA/MLRAres.pkl'
pickle.dump(md.res,open(path,'wb'))
print('Budget residuals are saved in ',path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.



Budget residuals are saved in  /local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/MLRL0res.pkl
Instructions for updating:
Use tf.cast instead.


/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/model_diagnostics.py:143: RuntimeWarning: divide by zero encountered in true_divide
  self.stats['r2'] = 1. - (self.stats['mse'] / self.stats['true_var'])
/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/model_diagnostics.py:149: RuntimeWarning: divide by zero encountered in true_divide
  self.stats['hor_r2'] = 1 - (self.stats['hor_mse'] / self.stats['hor_tvar'])


Stats are saved in  /local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/MLRAmd.pkl



Budget residuals are saved in  /local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/MLRAres.pkl
